In [4]:
%matplotlib widget
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import animation, rc
import matplotlib.ticker as ticker

from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
import ipywidgets as widgets

import time

import jupyros as jr
import rospy

from tf.transformations import euler_from_quaternion

import actionlib
import actionlib.msg
import rt2_assignment1.msg
from geometry_msgs.msg import Twist
from rt2_assignment1.srv import Command
from rosgraph_msgs.msg import Clock
from nav_msgs.msg import Odometry

#Control node initialization
rospy.init_node('jupyter_node')

# Global variable initialization
global linear_set, angular_set

linear_set = 0.0 
angular_set = 0.0


## Control Block

### Start and stop buttons

In [5]:
# Start and stop button definition
start  = Button(description='Start',layout=Layout(width='auto', align="center", grid_area='Start'),style=ButtonStyle(button_color='green'))
stop  = Button(description='Stop',layout=Layout(width='auto', grid_area='Stop'),style=ButtonStyle(button_color='red'))
output = widgets.Output()

#Node initialization
ui_client = rospy.ServiceProxy('/user_interface', Command)
action_service = actionlib.SimpleActionServer(
        '/go_to_point', rt2_assignment1.msg.Control2_1Action, auto_start=False)
action_client = actionlib.SimpleActionClient('/go_to_point', rt2_assignment1.msg.Control2_1Action)

# Stop the robot and initialize its status
ui_client("stop")
random_status = False

# Output of the buttons
def start_click(start):
    global random_status
    with output:
        ui_client("start")
        if random_status == True:
            print("I am already on my random way")
        random_status = True
        print('Start clicked')
        

def stop_click(stop):
    global random_status
    with output:
        ui_client("stop")
        if random_status == False:
            print("I was already stopped")
        random_status = False        
        print('Stop clicked')
        
    
# Callback definition
start.on_click(start_click)
stop.on_click(stop_click)


# Buttons display
buttons = GridBox(children=[start, stop],layout=Layout(width='50%',grid_template_rows='auto auto',grid_template_columns='50% 50%',grid_template_areas='''"Start Stop "'''))
widgets.VBox([buttons, output])


In [6]:
## Subscribe to the /go_to_point/result topic
hist_time = []
bar_y = [0, 0]

def result_callback(msg):
    if msg.result.reached == True:
        bar_y[1] = bar_y[1] + 1 
        hist_time.append(msg.header.stamp.secs - msg.status.goal_id.stamp.secs)
    else:
        bar_y[0] = bar_y[0] + 1 
    print(bar_y)
    print(hist_time)

# Subscribe to the /go_to_point/results topic
jr.subscribe('/go_to_point/result', rt2_assignment1.msg.Control2_1Action, result_callback)    

In [7]:
# Callback and data for the barplot with the result reached and cancelled
bar_fig = plt.figure()
ax_bar = bar_fig.add_subplot(111)
bar_x = ['Cancelled', 'Reached']
ax_bar.set_ylabel("Target summary")
rects = plt.bar(bar_x, bar_y, color='c')

def animate_rects(i):
    for rec, val in zip(rects, bar_y):
         rec.set_height(val)   
    ax_bar.set_ylim([0, max(bar_y)+1]) 
    return rects, 

# Create an animation function for the barplot
anim_reached = animation.FuncAnimation(bar_fig, animate_rects, blit=True)  

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# Callback and data for the histogram with the required time
hist_fig, ax_hist = plt.subplots()
ax_hist.set_xlabel("Time to target")
ax_hist.set_ylabel("Occurencies")
ax_hist.set_ylim([0, 1]) 

# Fixing bin edges
HIST_BINS = np.linspace(5, 40, 35)
n, _ = np.histogram(hist_time, HIST_BINS)


_, _, bar_container = ax_hist.hist(hist_time, HIST_BINS, lw=1,ec="yellow", fc="green", alpha=0.5)


def animate_hist(i):    
        n, _ = np.histogram(hist_time, HIST_BINS)
        ax_hist.hist(hist_time, HIST_BINS, lw=1,ec="yellow", fc="green", alpha=0.5)
        ax_hist.autoscale()
        

# Create an animation function for the histogram
anim_hist = animation.FuncAnimation(hist_fig, animate_hist, blit=True) 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Slider commands

In [9]:
pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)

msg = Twist()

a = widgets.FloatSlider(min=-1.0, max=1.0, description='Linear vel.')
display(a)

b = widgets.FloatSlider(min=-1.0, max=1.0, description='Angular vel.')
display(b)


def on_value_change(change):
    global msg, random_status, linear_set
    if random_status == False:
        msg.linear.x = change['new']
        linear_set = msg.linear.x
        pub.publish(msg)
    else:
        print("Command ignored, random position movement active")
    
def on_valueang_change(change):
    global msg, random_status, angular_set
    if random_status == False:
        msg.angular.z = change['new']
        angular_set = msg.angular.z 
        pub.publish(msg)
    else:
        print("Command ignored, random position movement active")

a.observe(on_value_change, names='value')
b.observe(on_valueang_change, names='value')


FloatSlider(value=0.0, description='Linear vel.', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='Angular vel.', max=1.0, min=-1.0)

Command ignored, random position movement active
Command ignored, random position movement active


### Control via buttons

In [10]:
up  = Button(description='W',layout=Layout(width='auto', align="center", grid_area='W'),style=ButtonStyle(button_color='gray'))
left  = Button(description='A',layout=Layout(width='auto', grid_area='A'),style=ButtonStyle(button_color='gray'))
stop_asd  = Button(description='Alt',layout=Layout(width='auto', grid_area='Alt'),style=ButtonStyle(button_color='red'))
right = Button(description='D',layout=Layout(width='auto', grid_area='D'),style=ButtonStyle(button_color='gray'))
down = Button(description='S',layout=Layout(width='auto', grid_area='S'),style=ButtonStyle(button_color='gray'))

# Output of the buttons for moving
def up_click(up):
    global random_status, linear_set, angular_set
    if random_status == True:
        ui_client("stop")
        print("Random go to point stopped, click again to start")
    elif random_status == False:
        msg.linear.x = linear_set
        msg.angular.z = 0.0
        pub.publish(msg)
        print("Go straight")     
        
def down_click(down):
    global random_status, linear_set, angular_set
    if random_status == True:
        ui_client("stop")
        print("Random go to point stopped, click again to start")
    elif random_status == False:
        msg.linear.x = -linear_set
        msg.angular.z = 0.0
        linear_set = msg.linear.x 
        angular_set = msg.angular.z 
        pub.publish(msg)
        print("Shrimp style")
        
def right_click(right):
    global random_status, linear_set, angular_set
    if random_status == True:
        ui_client("stop")
        print("Random go to point stopped, click again to start")
    elif random_status == False:
        msg.linear.x = linear_set
        msg.angular.z = angular_set
        pub.publish(msg)
        print("Turn right")
        
def left_click(left):
    global random_status, linear_set, angular_set
    if random_status == True:
        ui_client("stop")
        print("Random go to point stopped, click again to start")
    elif random_status == False:
        msg.linear.x = linear_set
        msg.angular.z = -angular_set
        pub.publish(msg)
        print("Turn left")

        
# Output of the button for stopping       
def stop_asd_click(stop_asd):
    global random_status, linear_set, angular_set
    if random_status == True:
        ui_client("stop")
        print("Random go to point stopped, manual mode active.")
        random_status = False
    elif random_status == False:
        msg.linear.x = 0.0
        msg.angular.z = 0.0
        linear_set = msg.linear.x 
        angular_set = msg.angular.z
        pub.publish(msg)
        print('Robot stopped')

# Callback definition
up.on_click(up_click)
down.on_click(down_click)
right.on_click(right_click)
left.on_click(left_click)
stop_asd.on_click(stop_asd_click)

# Buttons display
buttons = GridBox(children=[start, stop],layout=Layout(width='50%',grid_template_rows='auto auto',grid_template_columns='50% 50%',grid_template_areas='''"Start Stop "'''))
widgets.VBox([buttons, output])

GridBox(children=[up,  left, stop_asd, right, down],layout=Layout(width='50%',grid_template_rows='auto auto',grid_template_columns='33% 33% 33%',grid_template_areas='''" . W . ""A Alt D "".S ."'''))

GridBox(children=(Button(description='W', layout=Layout(grid_area='W', width='auto'), style=ButtonStyle(button…

## Display
### Visualize robot movement

In [13]:
# First set up the figures and the axis 
    # Robot position and orientation
fig, ax = plt.subplots()
ax.set_xlim(( -5, 5))
ax.set_ylim((-5, 5))
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Robot positon")

line, = ax.plot([], [], 'ro')

x_data = []
y_data = []
theta_data = []

#quiver initialization
Q = ax.quiver(1,1,1,1, alpha = 0)

def init():
    line.set_data([], [])
    return (line,)
    
def animate(i):
    global Q
    Q.remove()
    
    line.set_data(x_data, y_data)     
    
    X = x_data[-1] + 0.15 * np.cos(theta_data[-1])
    Y = y_data[-1] + 0.15 * np.sin(theta_data[-1])
    U = 0.2 * np.cos(theta_data[-1]) + np.cos(theta_data[-1])
    V = 0.2 * np.sin(theta_data[-1]) + np.sin(theta_data[-1])
    Q = ax.quiver(X, Y, U, V,scale = 1, color='k', pivot = 'tail', units='xy', alpha=0.6, edgecolor = 'gray')
    return (line, Q)

def odom_callback(msg):
    y_data.append(msg.pose.pose.position.y)
    x_data.append(msg.pose.pose.position.x)
    quaternion = (msg.pose.pose.orientation.x, msg.pose.pose.orientation.y,msg.pose.pose.orientation.z, msg.pose.pose.orientation.w)
    euler_angles = euler_from_quaternion(quaternion)
    theta_data.append(euler_angles[2]) 
      
                  
# Read robot odometry
jr.subscribe('/odom', Odometry, odom_callback)

# call the animator. blit=True means only re-draw the parts that have changed.)
anim_position = animation.FuncAnimation(fig, animate, init_func=init, blit=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Removing previous callback, only one redirection possible right now


### Velocity display

In [11]:
# Velocity feedback
fig_vel, ax_vel = plt.subplots()
ax_vel.set_xlim(0, 100)
ax_vel.set_ylim((-1.1, 1.1))
plt.title("Velocity check")
plt.xlabel("Time [s]")
plt.ylabel("Velocity")
# Array initilaization

time_vect = []
vel_target = []
vel_current = []
angular_target = []
angular_current = []

#Line initialization and color definition
    # linear velocity
line_vel_target, = ax_vel.plot([], [], 'ob', markersize='10')
line_vel_current, = ax_vel.plot([], [], 'or', markersize='5')
    # angular velocity
line_ang_target, = ax_vel.plot([], [], 'oc', markersize='10')
line_ang_current, = ax_vel.plot([], [], 'og', markersize='5')


def init_vel():
    # linear velocity lines
    line_vel_target.set_data([], [])
    line_vel_current.set_data([], [])    
    # angular velocity lines
    line_ang_target.set_data([], [])
    line_ang_current.set_data([], [])
    
    #return (line_vel_current, line_vel_target, ang_vel_current, ang_vel_target,)  
    return (line_vel_target, line_ang_target, line_vel_current, line_ang_current)

def animate_vel(i):   
    # linear velocity lines
    line_vel_target.set_data(time_vect, vel_target)   
    line_vel_current.set_data(time_vect, vel_current)  
    # angular velocity lines
    line_ang_target.set_data(time_vect, angular_target) 
    line_ang_current.set_data(time_vect, angular_current) 
    ax_vel.legend([line_vel_target, line_ang_target, line_vel_current, line_ang_current], ['Cmd. linear', 'Cmd. angular', 'Linear Vel.', 'Angular Vel.'], loc = 'lower left')
    time.sleep(1)
    return (line_vel_target, line_ang_target, line_vel_current, line_ang_current)

def vel_callback(msg_cmd):   
    
    time_vect.append(rospy.get_time())       
    vel_target.append(msg_cmd.linear.x)  
    angular_target.append(msg_cmd.angular.z)  
    x_min = max([0, max(time_vect) - 30]) 
    x_max = max([100, max(time_vect)])
    ax_vel.set_xlim( x_min, x_max )
    
def odom_speed_callback(msg_odom):       
    vel_current.append(msg_odom.twist.twist.linear.x) 
    angular_current.append(msg_odom.twist.twist.angular.z)

# Read robot velocity
jr.subscribe('cmd_vel', Twist, vel_callback)
# Read robot odometry
jr.subscribe('odom', Odometry, odom_speed_callback)

# Animation function for velocity check
anim_velocity = animation.FuncAnimation(fig_vel, animate_vel, init_func=init_vel, blit=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Removing previous callback, only one redirection possible right now
Removing previous callback, only one redirection possible right now
